```
Let's use hashindex to solve Wordle and see if it's fast!

Here is the game hashindex played.
```

![wordle game](wordle.png)

In [1]:
import string
from functools import partial
from hashindex import HashIndex
from words import all_wordle_words  # contains all 5-letter Wordle words as tuples of (word, frequency in English)

In [2]:
# Generate 5 functions to fetch the letter at each position in the word.
def char_at(pos, w):
    return w[0][pos]

def make_char_at_function(pos):
    p = partial(char_at, pos)
    p.__name__ = f'at_{pos}'
    return p

char_at = {pos: make_char_at_function(pos) for pos in range(5)}

In [3]:
# Generate 26 functions that count the occurrences of the specific letter.

def char_count(c, w):
    return w[0].count(c)

def make_char_count_function(letter):
    p = partial(char_count, letter)
    p.__name__ = f'has_{letter}'
    return p

char_count = {letter: make_char_count_function(letter) for letter in string.ascii_uppercase}

In [4]:
hi = HashIndex(all_wordle_words, on=list(char_at.values()) + list(char_count.values()))
hi.freeze()

In [5]:
ws = hi.find(
    match = {
    },
    exclude = {
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[array(['ABOUT', 1226734006], dtype=object),
 array(['OTHER', 978481319], dtype=object),
 array(['WHICH', 810514085], dtype=object),
 array(['THEIR', 782849411], dtype=object),
 array(['THERE', 701170205], dtype=object),
 array(['FIRST', 578161543], dtype=object),
 array(['WOULD', 572644147], dtype=object),
 array(['THESE', 541003982], dtype=object),
 array(['CLICK', 536746424], dtype=object),
 array(['PRICE', 501651226], dtype=object)]

In [6]:
ws = hi.find(
    match = {
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[array(['STUDY', 152978354], dtype=object),
 array(['UNTIL', 113090086], dtype=object),
 array(['STUFF', 81444510], dtype=object),
 array(['QUITE', 58777731], dtype=object),
 array(['UNITS', 56512308], dtype=object),
 array(['SUITE', 44829675], dtype=object),
 array(['TRUTH', 35345925], dtype=object),
 array(['TRUCK', 32136249], dtype=object),
 array(['TRULY', 21693553], dtype=object),
 array(['DUTCH', 19000979], dtype=object)]

In [7]:
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)
list(sorted(ws, key = lambda w: w[1], reverse=True))[:10]

[array(['HUTCH', 1373107], dtype=object),
 array(['CUTER', 152750], dtype=object),
 array(['MUTCH', 106606], dtype=object),
 array(['KUTCH', 67973], dtype=object),
 array(['TUTEE', 17303], dtype=object),
 array(['MUTER', 6358], dtype=object),
 array(['CUTCH', 6358], dtype=object)]

In [18]:
%%timeit -n 25 -r 5 
ws = hi.find(
    match = {
        char_count['S']: 0,
        char_count['A']: 0,
        char_count['B']: 0,
        char_count['O']: 0,
        char_count['Y']: 0,
        char_count['D']: 0,
        char_count['N']: 0,
        char_count['I']: 0,
        char_count['L']: 0,
        char_at[2]: 'T'
    },
    exclude = {
        char_count['U']: 0,
        char_count['T']: 0,
        char_at[0]: 'U',
        char_at[1]: 'T',
        char_at[2]: 'U',
        char_at[3]: 'U',
        char_at[4]: 'T',
    }
)

469 µs ± 45.2 µs per loop (mean ± std. dev. of 5 runs, 25 loops each)


```
Compare: find words by list comprehension
    
A frozen HashIndex is 10x faster or so. Not too shabby for such a small dataset!
```

In [21]:
%%timeit -n 25 -r 5 
z = [w for w in all_wordle_words if 
    w[0].count('S') == 0 and
    w[0].count('A') == 0 and
    w[0].count('B') == 0 and
    w[0].count('O') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('D') == 0 and
    w[0].count('N') == 0 and
    w[0].count('I') == 0 and
    w[0].count('L') == 0 and
    w[0][2] == 'T' and
    w[0].count('U') != 0 and
    w[0].count('T') != 0 and
    w[0][0] != 'U' and
    w[0][1] != 'T' and
    w[0][2] != 'U' and
    w[0][3] != 'U' and
    w[0][4] != 'T' 
]

4.21 ms ± 227 µs per loop (mean ± std. dev. of 5 runs, 25 loops each)


In [22]:
z = [w for w in all_wordle_words if 
    w[0].count('S') == 0 and
    w[0].count('A') == 0 and
    w[0].count('B') == 0 and
    w[0].count('O') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('Y') == 0 and
    w[0].count('D') == 0 and
    w[0].count('N') == 0 and
    w[0].count('I') == 0 and
    w[0].count('L') == 0 and
    w[0][2] == 'T' and
    w[0].count('U') != 0 and
    w[0].count('T') != 0 and
    w[0][0] != 'U' and
    w[0][1] != 'T' and
    w[0][2] != 'U' and
    w[0][3] != 'U' and
    w[0][4] != 'T' 
]
z

[('HUTCH', 1373107),
 ('CUTER', 152750),
 ('MUTCH', 106606),
 ('KUTCH', 67973),
 ('TUTEE', 17303),
 ('MUTER', 6358),
 ('CUTCH', 6358)]